# Solving a temperature-forecasting problem with GRUs

So far we've mainly discussed sequence data in the context of textual data, such as the IMDB dataset. However, sequence data is applicable to a wide range of problems beyond language processing. In the following examples, we'll look at a weather time series dataset collected at the weather station of the Max Planck Institute for Biogeochemistry in Jena, Germany.

This dataset includes 14 different variables such as air temperature, air pressure, humidity, wind direction and others, recorded every 10 minutes over several years. While the original data go back to 2003, our focus here is on the data from 2009 to 2016. Our goal is to construct a model that takes recent historical data (a few days' worth of data points) as input and predicts the air temperature 24 hours into the future.

You can obtain and decompress the data using the following instructions:

In [ ]:
#%%bash
#mkdir jena_climate
#cd jena_climate
#wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
#unzip jena_climate_2009_2016.csv.zip

By the way, the following code is a modified version of the code that can be found in [1]. That said, let us begin by importing some useful libraries and modules.

In [ ]:
import os
import numpy as np

from matplotlib import pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import Input
from keras.optimizers import RMSprop

Let's examine the data.

In [ ]:
data_dir = '/Users/dotero/Documents/TEC/Cursos/Bloque Integrador/2024/MA3001B/Code/jena_climate'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')
f = open(fname)
data = f.read()
f.close()
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
print(header)
print(len(lines))

We have a total of 420,551 lines of data (each line representing a time step with a date) and 14 weather-related measurements, plus the header. Now we will turn this data into a `NumPy` array.

In [ ]:
float_data = np.zeros((len(lines), len(header) - 1))

for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

Here is the plot of temperature (in degrees Celsius) over time. On this plot, you can clearly see the yearly periodicity of temperature.

In [ ]:
temp = float_data[:, 1]
plt.plot(range(len(temp)), temp)

## Formulation of the problem

The problem we will be solving with GRUs goes as follows: given data going as far back as `lookback` timesteps (a timestep is 10 minutes) and sampled every `steps` timesteps, can we predict the temperature in `delay` timesteps? We will use the following parameter values:

- `lookback`: 720—Observations will go back 5 days.
- `steps`: 6—Observations will be sampled at one data point per hour.
- `delay`: 144—Targets will be 24 hours in the future.

## Preparing the data

This task is straightforward because the data is already numerical, so no need for vectorization. However, each time series in the dataset operates on a different scale: for example, temperature typically ranges between -20 and +30, whereas atmospheric pressure, measured in mbar, hovers around 1,000. To deal with this, we'll normalize each time series independently, ensuring that they all have small values within a comparable scale.

We’ll preprocess the data by subtracting the mean of each timeseries and dividing by the standard deviation. We’re going to use the first 200,000 timesteps as training data, so we will compute the mean and standard deviation only on this fraction of the data.


In [ ]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

We will also need a `Python` generator that takes the current float data array and produces batches of historical data along with a target temperature for the future. Due to the high redundancy between samples in the dataset (where sample $n$ and sample $n+1$ share many timesteps), it's inefficient to allocate memory explicitly for each sample. Instead, you'll dynamically generate samples as needed using the original data.

The generator returns a tuple (`samples`, `targets`), where `samples` is a batch of input data and `targets` is the corresponding array of target temperatures. It takes the following arguments:

- `data`: The original array of floating-point data after normalization.
- `lookback`: How many timesteps back the input data should go.
- `delay`: How many timesteps in the future the target should be.
- `min_index` and `max_index`: Indices in the data array that delimit which time-steps to draw from. This is useful for keeping a segment of the data for validation and another for testing.
- `shuffle`: Whether to shuffle the samples or draw them in chronological order.
- `batch_size`: The number of samples per batch.
- `step`: The period, in timesteps, at which you sample data It’ll set it to 6 in order to draw one data point every hour.

In [ ]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

Now let's use the generic generator function to create three specific generators: one for training, one for validation, and one for testing. Each generator will focus on different time segments of the original data: the training generator will process the first 200,000 time steps, the validation generator will process the next 100,000 time steps, and the test generator will process the remaining data.

In [ ]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data, lookback=lookback, delay=delay, min_index=0,
                      max_index=200000, shuffle=True, step=step,
                      batch_size=batch_size)
val_gen = generator(float_data, lookback=lookback, delay=delay, min_index=200001,
                    max_index=300000, step=step, batch_size=batch_size)
test_gen = generator(float_data, lookback=lookback, delay=delay, min_index=300001,
                     max_index=None, step=step, batch_size=batch_size)

# How many steps to draw from val_gen in order to see the entire validation set
val_steps = (300000 - 200001 - lookback)
# How many steps to draw from test_gen in order to see the entire test set
test_steps = (len(float_data) - 300001 - lookback)

## A first recurrent baseline

We'll experiment with a recurrent sequence processing model, which is ideal for the type of data we're working with.

We are going to use the GRU layer developed by Chung et al. in 2014. Gated Recurrent Unit (GRU) layers work on the same principle as LSTMs, but are more streamlined and therefore less computationally intensive (although they may not offer the same level of representational capacity as LSTMs). This trade-off between computational efficiency and representational capacity is a common consideration in various aspects of machine learning.

By the way, note that we'll be using the **mean absolute error**, also knows as **MAE**, as our evaluation metric. This is one of many choices for regression and forecasting applications.

In [ ]:
model = Sequential()
model.add(Input(shape=(None, float_data.shape[-1])))
model.add(layers.GRU(32))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit(train_gen, steps_per_epoch=500, epochs=20,
                    validation_data=val_gen, validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

The validation MAE of ~0.265 (before overfitting) translates to a mean absolute error of 2.35 ̊C after denormalization. It's good, but we have a bit of a margin for improvement.

## Using recurrent dropout to fight overfitting

The training and validation curves clearly indicate that the model is experiencing overfitting, as evidenced by a significant divergence between their respective losses after a few epochs. You're already familiar with a traditional method to combat this issue: dropout, which randomly deactivates input units of a layer to break any incidental correlations in the training data that the layer might learn. However, applying dropout correctly in recurrent networks poses a non-trivial challenge. It's been long recognized that using dropout before a recurrent layer can hinder learning instead of aiding in regularization.

In 2015, Yarin Gal, as part of his PhD thesis on Bayesian deep learning, established the proper method for employing dropout in a recurrent network: applying the same dropout mask (i.e., the same pattern of deactivated units) at every timestep, rather than using a randomly varying dropout mask from timestep to timestep. Furthermore, to regularize the representations formed by the recurrent gates of layers like GRU and LSTM, a temporally consistent dropout mask should be applied to the inner recurrent activations of the layer (known as recurrent dropout). Employing the same dropout mask at each timestep enables the network to effectively propagate its learning errors over time, whereas a temporally random dropout mask would disrupt this error signal and impede the learning process.

In [ ]:
model = Sequential()
model.add(Input(shape=(None, float_data.shape[-1])))
model.add(layers.GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit_generator(train_gen, steps_per_epoch=500, epochs=10,
                                      validation_data=val_gen,
                                      validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

The overfitting problem has been solved, however, the best scores aren’t much lower than they were previously.

## Stacking recurrent layers

Since we've managed to address the overfitting issue but are now facing a performance bottleneck, it might be beneficial to enhance the network's capacity. Remember the universal machine-learning workflow, which suggests increasing network capacity until overfitting becomes the main challenge (assuming you're already employing basic techniques like dropout to mitigate overfitting). If you're not struggling with severe overfitting, it's likely that the network is under capacity.

Boosting network capacity usually involves adding more units to the layers or incorporating additional layers. One classic method to create more powerful recurrent networks is by stacking recurrent layers.

In Keras, to stack recurrent layers on top of each other, all intermediate layers should return their complete sequence of outputs (a 3D tensor) rather than just the output at the last timestep. This can be achieved by setting `return_sequences=True`.

In [ ]:
model = Sequential()
model.add(Input(shape=(None, float_data.shape[-1])))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5, return_sequences=True))
model.add(layers.GRU(64, activation='relu', dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit(train_gen, steps_per_epoch=500, epochs=10,
                              validation_data=val_gen,
                              validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

The inclusion of an extra layer does lead to a slight enhancement in the results, although the improvement is not particularly significant. Two key points can be inferred:

- Since we're still not experiencing severe overfitting, there is room to increase the size of the layers in an attempt to improve validation loss. However, this comes with a noticeable increase in computational demands.

- The addition of another layer did not result in a substantial improvement, indicating that we may be encountering diminishing returns from expanding the network's capacity further at this stage.

## References

[1] Chollet, Francois. *Deep learning with Python*. Simon and Schuster, 2021.